# Notebook to download the data for SEISMO-VRE  
Except where otherwise noted, this Jupyter Notebook is made available under the GPL-3.0 license. For details, please see the `LICENSE` file in the main directory.
# MATLAB Version

## Data list
### Analyses workflow 
1. [Atmospheric data from NASA](#Atmosphere-data)
2. [Ionosphere Swarm data from ESA](#Ionosphere-data)
3. [Geomagnetic indexes (Dst and ap)](#Geomagnetic-indexes).

Regarding the file `polos_igrf_swarm.dat`, it's necessary to update once a year inserting the geographic coorinates of North geomagnetic pole that can be retrieved at <a href="https://wdc.kugi.kyoto-u.ac.jp/poles/polesexp.html" target="_blank"> https://wdc.kugi.kyoto-u.ac.jp/poles/polesexp.html </a>.  

### Reconfigurable parameters
**Please, remember to update the global variables according to the earthquake you want to analyse in the main notebook**

In [1]:
% Global variables initialization

% Earthquake origin time
EQ_year = 2016; EQ_month=8; EQ_day=24; EQ_hour=1; EQ_minute=36; EQ_second = 32;
%Epicenter latitude and longitude
epilat = 42.6980; epilon = 13.2340;
%Mainshock magnitude
EQ_mag = 6.5;
EQ_depth = 10;
Fault_strike = 153; %Here is inserted a mean strike of the two main faults: Amatrice 155 and Norcia 151
%This strike must be a positive value
day_before = 8*30;
day_after = 0;

## Atmosphere data

The atmospheric analysis uses the MERRA2 climatological archive data from NASA (doi: <a href="https://doi.org/10.5067/KLICLTZ8EM9D"> 10.5067/KLICLTZ8EM9D</a>).  

**If you need to change the case study, you must delete the file `Atmospheric_data_from_MERRA2.mat`**. Otherwise, the analysis could potentially mix data from different events.

In this session the data from MERRA-2 Climatological archive will be downloaded and analysed
**Important note:**  
The MERRA-2 data are open but require a free account on EarthData Nasa portal. The following line will ask you your username (generally your registration email) and your password in order to download the data.

In order to mask the password the Matlab tool logindlg has been used. Proper credit to: Jeremy Smith (2025). login (https://www.mathworks.com/matlabcentral/fileexchange/8499-login), MATLAB Central File Exchange. Retrieved January 25, 2025.

The following cell reports the function called *"logindlg"* by Jeremy Smith that permits the insertion of ones's own credentials masking the password. 


In [2]:
% Credit of this cell and function to Jeremy Smith, https://www.mathworks.com/matlabcentral/fileexchange/8499-login
% LOGINDLG   Dialog for visually secure login.
%   Examples:
%       [login password] = logindlg('Title','Login Title');  % Returns the login and
%          password with the dialog title 'Login Title'.
%
%       password = logindlg;  % Returns only the password with a default
%          dialog title.
%
%       password = logindlg('Password','only');  % Displays only a password
%          edit box and returns the password.
%
%       password = logindlg('Title','Login Title','Password','only');
%          % Creates a password only dialog with 'Login Title' as the
%          dialog title.
%
%
%
%
% Author: Jeremy Smith
% Date: September 24, 2005
% Last Edit: January 28, 2010
% Version: 1.3
% Tested on: Matlab 7.0.4.365 (R14) Service Pack 2, Matlab 7.1 SP 3, and
%       Matlab 7.4.0.287 (R2007a)
% Description: custom login dialog because Matlab doesn't have an option
%       for characters in an edit field to be replaced by asterisks
%       (password security).
%
%       Note:  On very slow computers the first few password characters may 
%       have a delay before they are converted to asterisks.

% Changelist:
%   1.3: -Pressing the Escape key with focus anywhere on the dialog will
%            now cancel the dialog box
%        -Typo correction
%        -Escape no longer creates an empty character in the password field
%   1.2: -Tab no longer triggers the OK button in the password box
%        -Improved the script help
%        -Removed horizontal alignment from buttons
%        -Added the option to display only the password box
%   1.1: -Added positioning code so it'll display in the center of the screen
%        -If only one output is specified the password will be returned
%            instead of the login as in Version 1.0
%        -Escape will not only close the dialog if neither edit box is active
%        -When the dialog appears the first edit box will be active
%        -Added a few more comments
%        -Removed the clc, it was left in by mistake in Version 1.0

function [varargout]=logindlg(varargin)

% Number of inputs check
if nargin ==  0 || nargin == 2 || nargin == 4
else
    error('Incorrect number of input arguments.')
end

% Input Type Check
for i=1:1:length(varargin)
    if ~ischar(varargin{i})
        error('Inputs must be strings.')
    end
end

% Title Option
if nargin == 0
    Title = 'Login';
elseif nargin == 2 && ~isempty(strmatch('title',lower(varargin)))
    Title = varargin{2};
elseif nargin == 2 && isempty(strmatch('title',lower(varargin)))
    Title = 'Login';
elseif nargin == 4 && ~isempty(strmatch('title',lower(varargin)))
    S = strmatch('title',lower(varargin));
    if S == 1
        Title = varargin{2};
    elseif S == 3
        Title = varargin{4};
    else
        error('Invalid title.')
    end
else
    error('Invalid title.')
end

% Password Option
if nargin == 0
    Pass = 0;
elseif nargin == 2 && ~isempty(strmatch('password',lower(varargin{1}))) && ~isempty(strmatch('only',lower(varargin{2})))
    Pass = 1;
elseif nargin == 4 && ~isempty(strmatch('password',lower(varargin))) && ~isempty(strmatch('only',lower(varargin)))
    P = strmatch('password',lower(varargin));
    O = strmatch('only',lower(varargin));
    if P == 1 && O == 2
        Pass = 1;
    elseif P == 3 && O == 4
        Pass = 1;
    end
elseif nargin == 2 && isempty(strmatch('password',lower(varargin))) == 1
    Pass = 0;
else
    error('Invalid password option.')
end

% Output Error Check
if nargout > 1 && Pass == 1 || nargout > 2
    error('Too many output arguments.')
end

% Get Properties
Color = get(0,'DefaultUicontrolBackgroundcolor');

% Determine the size and position of the login interface
if Pass == 0
    Height = 9.5;
else
    Height = 5.5;
end
set(0,'Units','characters')
Screen = get(0,'screensize');
Position = [Screen(3)/2-17.5 Screen(4)/2-4.75 35 Height];
set(0,'Units','pixels')

% Create the GUI
gui.main = dialog('HandleVisibility','on',...
    'IntegerHandle','off',...
    'Menubar','none',...
    'NumberTitle','off',...
    'Name','Login',...
    'Tag','logindlg',...
    'Color',Color,...
    'Units','characters',...
    'Userdata','logindlg',...
    'Position',Position);

% Set the title
if ischar(Title) == 1
    set(gui.main,'Name',Title,'Closerequestfcn',{@Cancel,gui.main},'Keypressfcn',{@Escape})
end

% Texts
if Pass == 0
    gui.login_text = uicontrol(gui.main,'Style','text','FontSize',8,'HorizontalAlign','left','Units','characters','String','Login','Position',[1 7.65 20 1]);
end
gui.password_text = uicontrol(gui.main,'Style','text','FontSize',8,'HorizontalAlign','left','Units','characters','String','Password','Position',[1 4.15 20 1]);

% Edits
if Pass == 0
    gui.edit1 = uicontrol(gui.main,'Style','edit','FontSize',8,'HorizontalAlign','left','BackgroundColor','white','Units','characters','String','','Position',[1 6.02 33 1.7],'KeyPressfcn',{@Escape});
end
gui.edit2 = uicontrol(gui.main,'Style','edit','FontSize',8,'HorizontalAlign','left','BackgroundColor','white','Units','characters','String','','Position',[1 2.52 33 1.7],'KeyPressfcn',{@KeyPress_Function,gui.main},'Userdata','');

% Buttons
gui.OK = uicontrol(gui.main,'Style','push','FontSize',8,'Units','characters','String','OK','Position',[12 .2 10 1.7],'Callback',{@OK,gui.main},'KeyPressfcn',{@Escape});
gui.Cancel = uicontrol(gui.main,'Style','push','FontSize',8,'Units','characters','String','Cancel','Position',[23 .2 10 1.7],'Callback',{@Cancel,gui.main},'KeyPressfcn',{@Escape});

setappdata(0,'logindlg',gui) % Save handle data
setappdata(gui.main,'Check',0) % Error check setup. If Check remains 0 an empty cell array will be returned

if Pass == 0
    uicontrol(gui.edit1) % Make the first edit box active
else
    uicontrol(gui.edit2)  % Make the second edit box active if the first isn't present
end

% Pause the GUI and wait for a button to be pressed
uiwait(gui.main)

Check = getappdata(gui.main,'Check'); % Check to see if a button was pressed

% Format output
if Check == 1
    if Pass == 0
        Login = get(gui.edit1,'String');
    end
    Password = get(gui.edit2,'Userdata');
    
    if nargout == 1 % If only one output specified output Password
        varargout(1) = {Password};
    elseif nargout == 2 % If two outputs specified output both Login and Password
        varargout(1) = {Login};
        varargout(2) = {Password};
    end
else % If OK wasn't pressed output nothing
    if nargout == 1
        varargout(1) = {[]};
    elseif nargout == 2
        varargout(1) = {[]};
        varargout(2) = {[]};
    end
end

delete(gui.main) % Close the GUI
setappdata(0,'logindlg',[]) % Erase handles from memory

%% Hide Password
function KeyPress_Function(h,eventdata,fig)
% Function to replace all characters in the password edit box with
% asterisks
password = get(h,'Userdata');
key = get(fig,'currentkey');

switch key
    case 'backspace'
        password = password(1:end-1); % Delete the last character in the password
    case 'return'  % This cannot be done through callback without making tab to the same thing
        gui = getappdata(0,'logindlg');
        OK([],[],gui.main);
    case 'tab'  % Avoid tab triggering the OK button
        gui = getappdata(0,'logindlg');
        uicontrol(gui.OK);
    case 'escape'
        % Close the login dialog
        Escape(fig,[])
    otherwise
        password = [password get(fig,'currentcharacter')]; % Add the typed character to the password
end

SizePass = size(password); % Find the number of asterisks
if SizePass(2) > 0
    asterisk(1,1:SizePass(2)) = '*'; % Create a string of asterisks the same size as the password
    set(h,'String',asterisk) % Set the text in the password edit box to the asterisk string
else
    set(h,'String','')
end

set(h,'Userdata',password) % Store the password in its current state

%% Cancel
function Cancel(h,eventdata,fig)
uiresume(fig)

%% OK
function OK(h,eventdata,fig)
% Set the check and resume
setappdata(fig,'Check',1)
uiresume(fig)

%% Escape
function Escape(h,eventdata)
% Close the login if the escape button is pushed and neither edit box is
% active
fig = h;
while ~strcmp(get(fig,'Type'),'figure')
    fig = get(fig,'Parent');
end
key = get(fig,'currentkey');

if isempty(strfind(key,'escape')) == 0
    Cancel([],[],fig)
end

The following line acquires your username and password of EarthData NASA account (free registration). This is necessary to download the MERRA-2 data. In case you already have the data locally, you can skip the following two cells.  
To improve the speed of download of the data, which may require up to 5 days, it's suggested to run the following cells inside Matlab Interface/command line.

In [3]:
%Acquisition of your EarthData NASA account username and password (free registration).
[User_ED pwd_ED] = logindlg('Please insert your username and password for your EarthData account','Login Title');

In [4]:
% Section to download the necessary data from NASA - MERRA2

%set the basic parameters to connect with NASA server
options_for_NASA = weboptions('Username', User_ED,'Password', pwd_ED, 'Timeout', 30000);
%Calculate the EQ time and research area to download the data
EQ_time = datenum([EQ_year, EQ_month, EQ_day, EQ_hour, EQ_minute, EQ_second]);
radius_Dobrovolsky = 10.^(0.43*EQ_mag)/111;
min_lat = round(epilat - radius_Dobrovolsky/sqrt(2));
max_lat = round(epilat + radius_Dobrovolsky/sqrt(2));
min_lon = round(epilon - radius_Dobrovolsky/sqrt(2));
max_lon = round(epilon + radius_Dobrovolsky/sqrt(2));
%Calculate the local midnight at earthquake longitude
local_midnight = - epilon/15;
% search for the closest UT clock time to the local midnight
UT_time_of_data = round(local_midnight);
if UT_time_of_data >= 24
    UT_time_of_data = UT_time_of_data - 24;
end
if UT_time_of_data < 0
    UT_time_of_data = UT_time_of_data + 24;
end
% set the corresponding index for the specific hour
index_of_time_data = UT_time_of_data + 1; %The index is increased of a unit as Matlab starts to count from "1"

% set the time interval limits. This limit is for the day/month
time_start = EQ_time - day_before;
time_end = EQ_time + day_after;
% Search for the last year to be downloaded. In case the earthquake occurred before the present year, it is the previous year.
[year_end1, month_end1, day_end1] = datevec(time_end);
[year_end, ~] = datevec(now); %2024;
if EQ_year < year_end
    year_end = year_end -1; %Stop the analysis the year before the present one.
end
% calculate the total number of analysed day for each year
number_of_analysed_days = day_after + day_before + 1;
% Calculate the start year, considering that the dataset started in 1980, but if the interval would extend in 1979 the starting year is considered 1981
if (datenum([1980 month_end1 day_end1]) - number_of_analysed_days) < datenum([1980 1 1])
    year_start = 1981;
else
    year_start = 1980;
end
[~, month_start, day_start] = datevec(time_start);
% set the first day to be downloaded
time_start = datenum([year_start, month_start, day_start]);
% set the last day to be downloaded
time_end = datenum([year_end, month_end1, day_end1]);
number_of_analysed_days = time_end - time_start + 1;
% check if some data has already been downloaded before and load them
if exist(fullfile('data', 'Atmospheric_data_from_MERRA2.mat')) > 0
    load(fullfile('data', 'Atmospheric_data_from_MERRA2.mat'));
else
% otherwise the variable to store the atmospheric data and their timestamp are created
    TCWV = nan(number_of_analysed_days, 1);
    TCWV_time = nan(number_of_analysed_days, 1);
    SO2 = nan(number_of_analysed_days, 1);
    SO2_time = nan(number_of_analysed_days, 1);
    DMS = nan(number_of_analysed_days, 1);
    DMS_time = nan(number_of_analysed_days, 1);
    AOT = nan(number_of_analysed_days, 1);
    AOT_time = nan(number_of_analysed_days, 1);
    CO = nan(number_of_analysed_days, 1);
    CO_time = nan(number_of_analysed_days, 1);
    TO3 = nan(number_of_analysed_days, 1);
    TO3_time = nan(number_of_analysed_days, 1);
    SAT = nan(number_of_analysed_days, 1);
    SAT_time = nan(number_of_analysed_days, 1);
    HUM = nan(number_of_analysed_days, 1);
    HUM_time = nan(number_of_analysed_days, 1);
    SLHF = nan(number_of_analysed_days, 1);
    SLHF_time = nan(number_of_analysed_days, 1);
    RAIN = nan(number_of_analysed_days, 1);
    RAIN_time = nan(number_of_analysed_days, 1);
end

% creation of a vector with the day that must be checked/downloaded, i.e., the day necessary for the atmospheric analysis
k = 1;
list_of_days = nan(number_of_analysed_days, 1);
for year_tba = year_start:year_end
    time_start_1 = datenum([year_tba EQ_month EQ_day]) - day_before-1;
    time_end_1 = datenum([year_tba EQ_month EQ_day]) + day_after;
    for day_tba = time_start_1:time_end_1
        list_of_days(k,1) = day_tba;
        k = k + 1;
    end
end
list_of_days = list_of_days(1:k-1);

% cycle all the days.
% Firstly, the existance of the specific variable is checked, as it could be already downloaded otherwise the corresponding request to the NASA archive is performed 
k = 1;
for i_day = 1:size(list_of_days,1)
    day_to_be_analyed=list_of_days(i_day); 
    [year_tba, month_tba, day_tba] = datevec(day_to_be_analyed)
    % check of TCWV variable of the specific day exist or not
    if sum(TCWV_time == day_to_be_analyed) == 0
        % search for the first available row inside the vector
        index_nan = find(isnan(TCWV_time));
        if size(index_nan,1) == 0
            index_nan = size(TCWV_time,1) + 1;
        else
            index_nan = index_nan(1);
        end
        %The version of MERRA-2 changed among the time, so the following filename must be followed.
        % MERRA2_100.*.19800101.nc4
        % MERRA2_200.*.19920101.nc4
        % MERRA2_300.*.20010101.nc4
        % MERRA2_400.*.20110101.nc4
        if day_to_be_analyed < datenum([1992 1 1])
            version_f = 'MERRA2_100';
        elseif day_to_be_analyed < datenum([2001 1 1])
            version_f = 'MERRA2_200';
        elseif day_to_be_analyed < datenum([2011 1 1])
            version_f = 'MERRA2_300';
        elseif (day_to_be_analyed >= datenum([2020 9 1]) && day_to_be_analyed <= datenum([2020 9 30])) || (day_to_be_analyed >= datenum([2021 6 1]) && day_to_be_analyed <= datenum([2021 9 30]))
            version_f = 'MERRA2_401';
        else
            version_f = 'MERRA2_400';
        end
        % The URL to download the data is constructed
        weburl_to_retrive_the_data = ['https://goldsmr4.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FMERRA2%2FM2I1NXINT.5.12.4%2F', num2str(year_tba), '%2F', num2str(month_tba, '%02.0f'), '%2F', version_f, '.inst1_2d_int_Nx.', num2str(year_tba), num2str(month_tba, '%02.0f'), num2str(day_tba, '%02.0f'), '.nc4&VERSION=1.02&SERVICE=L34RS_MERRA2&DATASET_VERSION=5.12.4&LABEL=', version_f, '.inst1_2d_int_Nx.', num2str(year_tba), num2str(month_tba, '%02.0f'), num2str(day_tba, '%02.0f'), '.SUB.nc&BBOX=', num2str(min_lat),'%2C', num2str(min_lon), '%2C', num2str(max_lat), '%2C', num2str(max_lon), '&FORMAT=bmM0Lw&VARIABLES=TQV&SHORTNAME=M2I1NXINT'];
        % Data are downloaded from the server
        TCVW_p = webread(weburl_to_retrive_the_data, options_for_NASA);
        % The data are temporarly written in a file test_MERRA.nc
        file_output = fopen('test_MERRA.nc', 'w');
        fwrite(file_output, TCVW_p);
        fclose(file_output);
        %The data are read from the temporarly file test_MERRA.nc
        TCVW_1 = ncread('test_MERRA.nc', 'TQV');
        % data and time are stored in the proper variable (TCWV and TCWV_time)
        TCWV(index_nan,1) = mean(reshape(TCVW_1(:,:,index_of_time_data), size(TCVW_1,1)*size(TCVW_1,2), 1));
        TCWV_time(index_nan,1) = day_to_be_analyed;
        % Every 15 days the data are saved. In this way, in case of failure data lost is reduced. Data are not written for each day otehrwise the cycle is too slow for continuos writing to HDD 
        if mod(k,15) == 0 %save downloaded data every 15 timesteps to protect against eventual website failure
            save(fullfile('data', 'Atmospheric_data_from_MERRA2.mat'), 'TCWV', 'TCWV_time', 'SO2', 'SO2_time', 'DMS', 'DMS_time', 'AOT', 'AOT_time', 'CO', 'CO_time', 'TO3', 'TO3_time',  'SAT', 'SAT_time', 'HUM', 'HUM_time', 'SLHF', 'SLHF_time', 'RAIN', 'RAIN_time');
        end
    end

% All the previous procedure is repeated for SO2, DMS and Aerosol (AOT) parameters
    if sum(SO2_time == day_to_be_analyed) == 0
        index_nan = find(isnan(SO2_time));
        if size(index_nan,1) == 0
            index_nan = size(SO2_time,1) + 1;
        else
            index_nan = index_nan(1);
        end
        if day_to_be_analyed < datenum([1992 1 1])
            version_f = 'MERRA2_100';
        elseif day_to_be_analyed < datenum([2001 1 1])
            version_f = 'MERRA2_200';
        elseif day_to_be_analyed < datenum([2011 1 1])
            version_f = 'MERRA2_300';
        elseif (day_to_be_analyed >= datenum([2020 9 1]) && day_to_be_analyed <= datenum([2020 9 30])) || (day_to_be_analyed >= datenum([2021 6 1]) && day_to_be_analyed <= datenum([2021 9 30]))
            version_f = 'MERRA2_401';
        else
            version_f = 'MERRA2_400';
        end
        weburl_to_retrive_the_data = ['https://goldsmr4.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FMERRA2%2FM2T1NXAER.5.12.4%2F', num2str(year_tba), '%2F', num2str(month_tba, '%02.0f'), '%2F', version_f, '.tavg1_2d_aer_Nx.', num2str(year_tba), num2str(month_tba, '%02.0f'), num2str(day_tba, '%02.0f'), '.nc4&BBOX=', num2str(min_lat),'%2C', num2str(min_lon), '%2C', num2str(max_lat), '%2C', num2str(max_lon), '&SERVICE=L34RS_MERRA2&VERSION=1.02&LABEL=', version_f, '.tavg1_2d_aer_Nx.', num2str(year_tba), num2str(month_tba, '%02.0f'), num2str(day_tba, '%02.0f'), '.SUB.nc&SHORTNAME=M2T1NXAER&TIME=', num2str(year_tba), '-', num2str(month_tba, '%02.0f'),'-', num2str(day_tba, '%02.0f'), 'T', num2str(UT_time_of_data, '%02.0f'),'%3A30%3A00&DATASET_VERSION=5.12.4&VARIABLES=DMSSMASS%2CSO2SMASS%2CTOTEXTTAU&FORMAT=nc4%2F'];
        SO2_p = webread(weburl_to_retrive_the_data, options_for_NASA);
        file_output = fopen('test_MERRA.nc', 'w');
        fwrite(file_output, SO2_p);
        fclose(file_output);
        SO2_1 = ncread('test_MERRA.nc', 'SO2SMASS');
        DMS_1 = ncread('test_MERRA.nc', 'DMSSMASS');
        AOT_1 = ncread('test_MERRA.nc', 'TOTEXTTAU');
        SO2(index_nan,1) = mean(reshape(SO2_1(:,:,index_of_time_data), size(SO2_1,1)*size(SO2_1,2), 1));
        SO2_time(index_nan,1) = day_to_be_analyed;
        DMS(index_nan,1) = mean(reshape(DMS_1(:,:,index_of_time_data), size(DMS_1,1)*size(DMS_1,2), 1));
        DMS_time(index_nan,1) = day_to_be_analyed;
        AOT(index_nan,1) = mean(reshape(AOT_1(:,:,index_of_time_data), size(AOT_1,1)*size(AOT_1,2), 1));
        AOT_time(index_nan,1) = day_to_be_analyed;
        if mod(k,15) == 0 %save downloaded data every 15 timesteps to protect against eventual website failure
            save(fullfile('data', 'Atmospheric_data_from_MERRA2.mat'), 'TCWV', 'TCWV_time', 'SO2', 'SO2_time', 'DMS', 'DMS_time', 'AOT', 'AOT_time', 'CO', 'CO_time', 'TO3', 'TO3_time',  'SAT', 'SAT_time', 'HUM', 'HUM_time', 'SLHF', 'SLHF_time', 'RAIN', 'RAIN_time');
        end
    end

% All the previous procedure is repeated for CO and Total Column Ozon (TO3) parameters
    if sum(CO_time == day_to_be_analyed) == 0
    index_nan = find(isnan(CO_time));
        if size(index_nan,1) == 0
            index_nan = size(CO_time,1) + 1;
        else
            index_nan = index_nan(1);
        end
        if day_to_be_analyed < datenum([1992 1 1])
            version_f = 'MERRA2_100';
        elseif day_to_be_analyed < datenum([2001 1 1])
            version_f = 'MERRA2_200';
        elseif day_to_be_analyed < datenum([2011 1 1])
            version_f = 'MERRA2_300';
        elseif (day_to_be_analyed >= datenum([2020 9 1]) && day_to_be_analyed <= datenum([2020 9 30])) || (day_to_be_analyed >= datenum([2021 6 1]) && day_to_be_analyed <= datenum([2021 9 30]))
            version_f = 'MERRA2_401';
        else
            version_f = 'MERRA2_400';
        end
        weburl_to_retrive_the_data = ['https://goldsmr4.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FMERRA2%2FM2T1NXCHM.5.12.4%2F', num2str(year_tba), '%2F', num2str(month_tba, '%02.0f'), '%2F', version_f, '.tavg1_2d_chm_Nx.', num2str(year_tba), num2str(month_tba, '%02.0f'), num2str(day_tba, '%02.0f'), '.nc4&FORMAT=bmM0Lw&VERSION=1.02&LABEL=', version_f, '.tavg1_2d_chm_Nx.', num2str(year_tba), num2str(month_tba, '%02.0f'), num2str(day_tba, '%02.0f'), '.SUB.nc&SHORTNAME=M2T1NXCHM&TIME=', num2str(year_tba), '-', num2str(month_tba, '%02.0f'),'-', num2str(day_tba, '%02.0f'), 'T', num2str(UT_time_of_data, '%02.0f'),'%3A00%3A00&DATASET_VERSION=5.12.4&BBOX=', num2str(min_lat),'%2C', num2str(min_lon), '%2C', num2str(max_lat), '%2C', num2str(max_lon), '&VARIABLES=COSC%2CTO3&SERVICE=L34RS_MERRA2'];
        CO_p = webread(weburl_to_retrive_the_data, options_for_NASA);
        file_output = fopen('test_MERRA.nc', 'w');
        fwrite(file_output, CO_p);
        fclose(file_output);
        CO_1 = ncread('test_MERRA.nc', 'COSC');
        TO3_1 = ncread('test_MERRA.nc', 'TO3');
        CO(index_nan,1) = mean(reshape(CO_1(:,:,index_of_time_data), size(CO_1,1)*size(CO_1,2), 1));
        CO_time(index_nan,1) = day_to_be_analyed;
        TO3(index_nan,1) = mean(reshape(TO3_1(:,:,index_of_time_data), size(TO3_1,1)*size(TO3_1,2), 1));
        TO3_time(index_nan,1) = day_to_be_analyed;
        
        if mod(k,15) == 0 %save downloaded data every 15 timesteps to protect against eventual website failure
            save(fullfile('data', 'Atmospheric_data_from_MERRA2.mat'), 'TCWV', 'TCWV_time', 'SO2', 'SO2_time', 'DMS', 'DMS_time', 'AOT', 'AOT_time', 'CO', 'CO_time', 'TO3', 'TO3_time',  'SAT', 'SAT_time', 'HUM', 'HUM_time', 'SLHF', 'SLHF_time', 'RAIN', 'RAIN_time');
        end
    end

% All the previous procedure is repeated for Surface Air Temperature (SAT) and Humidity parameters
    if sum(SAT_time == day_to_be_analyed) == 0
        index_nan = find(isnan(SAT_time));
        if size(index_nan,1) == 0
            index_nan = size(SAT_time,1) + 1;
        else
            index_nan = index_nan(1);
        end
        if day_to_be_analyed < datenum([1992 1 1])
            version_f = 'MERRA2_100';
        elseif day_to_be_analyed < datenum([2001 1 1])
            version_f = 'MERRA2_200';
        elseif day_to_be_analyed < datenum([2011 1 1])
            version_f = 'MERRA2_300';
        elseif (day_to_be_analyed >= datenum([2020 9 1]) && day_to_be_analyed <= datenum([2020 9 30])) || (day_to_be_analyed >= datenum([2021 6 1]) && day_to_be_analyed <= datenum([2021 9 30]))
            version_f = 'MERRA2_401';
        else
            version_f = 'MERRA2_400';
        end
        weburl_to_retrive_the_data = ['https://goldsmr4.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FMERRA2%2FM2I1NXLFO.5.12.4%2F', num2str(year_tba), '%2F', num2str(month_tba, '%02.0f'), '%2F', version_f, '.inst1_2d_lfo_Nx.', num2str(year_tba), num2str(month_tba, '%02.0f'), num2str(day_tba, '%02.0f'), '.nc4&VERSION=1.02&SERVICE=L34RS_MERRA2&BBOX=', num2str(min_lat),'%2C', num2str(min_lon), '%2C', num2str(max_lat), '%2C', num2str(max_lon), '&DATASET_VERSION=5.12.4&SHORTNAME=M2I1NXLFO&TIME=', num2str(year_tba), '-', num2str(month_tba, '%02.0f'),'-', num2str(day_tba, '%02.0f'), 'T', num2str(UT_time_of_data, '%02.0f'),'%3A00%3A00&LABEL=', version_f, '.inst1_2d_lfo_Nx.', num2str(year_tba), num2str(month_tba, '%02.0f'), num2str(day_tba, '%02.0f'), '.SUB.nc&VARIABLES=QLML%2CTLML&FORMAT=nc4%2F'];
        SAT_p = webread(weburl_to_retrive_the_data, options_for_NASA);
        file_output = fopen('test_MERRA.nc', 'w');
        fwrite(file_output, SAT_p);
        fclose(file_output);
        SAT_1 = ncread('test_MERRA.nc', 'TLML');
        HUM_1 = ncread('test_MERRA.nc', 'QLML');
        SAT(index_nan,1) = mean(reshape(SAT_1(:,:,index_of_time_data), size(SAT_1,1)*size(SAT_1,2), 1));
        SAT_time(index_nan,1) = day_to_be_analyed;
        HUM(index_nan,1) = mean(reshape(HUM_1(:,:,index_of_time_data), size(HUM_1,1)*size(HUM_1,2), 1));
        HUM_time(index_nan,1) = day_to_be_analyed;
        
        if mod(k,15) == 0 %save downloaded data every 15 timesteps to protect against eventual website failure
            save(fullfile('data', 'Atmospheric_data_from_MERRA2.mat'), 'TCWV', 'TCWV_time', 'SO2', 'SO2_time', 'DMS', 'DMS_time', 'AOT', 'AOT_time', 'CO', 'CO_time', 'TO3', 'TO3_time',  'SAT', 'SAT_time', 'HUM', 'HUM_time', 'SLHF', 'SLHF_time', 'RAIN', 'RAIN_time');
        end
    end

% All the previous procedure is repeated for Rain and Surface Latent Heat Flux (SLHF) parameters
    if sum(RAIN_time == day_to_be_analyed) == 0
    % if isnan(RAIN_time(k)) == 1
        index_nan = find(isnan(RAIN_time));
        if size(index_nan,1) == 0
            index_nan = size(RAIN_time,1) + 1;
        else
            index_nan = index_nan(1);
        end
        if day_to_be_analyed < datenum([1992 1 1])
            version_f = 'MERRA2_100';
        elseif day_to_be_analyed < datenum([2001 1 1])
            version_f = 'MERRA2_200';
        elseif day_to_be_analyed < datenum([2011 1 1])
            version_f = 'MERRA2_300';
        elseif (day_to_be_analyed >= datenum([2020 9 1]) && day_to_be_analyed <= datenum([2020 9 30])) || (day_to_be_analyed >= datenum([2021 6 1]) && day_to_be_analyed <= datenum([2021 9 30]))
            version_f = 'MERRA2_401';
        else
            version_f = 'MERRA2_400';
        end
        weburl_to_retrive_the_data = ['https://goldsmr4.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FMERRA2%2FM2T1NXFLX.5.12.4%2F', num2str(year_tba), '%2F', num2str(month_tba, '%02.0f'), '%2F', version_f, '.tavg1_2d_flx_Nx.', num2str(year_tba), num2str(month_tba, '%02.0f'), num2str(day_tba, '%02.0f'), '.nc4&VARIABLES=EFLUX%2CPRECTOT&VERSION=1.02&SERVICE=L34RS_MERRA2&DATASET_VERSION=5.12.4&BBOX=', num2str(min_lat),'%2C', num2str(min_lon), '%2C', num2str(max_lat), '%2C', num2str(max_lon), '&FORMAT=bmM0Lw&SHORTNAME=M2T1NXFLX&TIME=', num2str(year_tba), '-', num2str(month_tba, '%02.0f'),'-', num2str(day_tba, '%02.0f'), 'T', num2str(UT_time_of_data, '%02.0f'),'%3A00%3A00&LABEL=', version_f, '.tavg1_2d_flx_Nx.', num2str(year_tba), num2str(month_tba, '%02.0f'), num2str(day_tba, '%02.0f'), '.SUB.nc'];
        SLHF_p = webread(weburl_to_retrive_the_data, options_for_NASA);
        file_output = fopen('test_MERRA.nc', 'w');
        fwrite(file_output, SLHF_p);
        fclose(file_output);
        SLHF_1 = ncread('test_MERRA.nc', 'EFLUX');
        RAIN_1 = ncread('test_MERRA.nc', 'PRECTOT');
        SLHF(index_nan,1) = mean(reshape(SLHF_1(:,:,index_of_time_data), size(SLHF_1,1)*size(SLHF_1,2), 1));
        SLHF_time(index_nan,1) = day_to_be_analyed;
        RAIN(index_nan,1) = mean(reshape(RAIN_1(:,:,index_of_time_data), size(RAIN_1,1)*size(RAIN_1,2), 1));
        RAIN_time(index_nan,1) = day_to_be_analyed;
        
        if mod(k,15) == 0 %save downloaded data every 15 timesteps to protect against eventual website failure
            save(fullfile('data', 'Atmospheric_data_from_MERRA2.mat'), 'TCWV', 'TCWV_time', 'SO2', 'SO2_time', 'DMS', 'DMS_time', 'AOT', 'AOT_time', 'CO', 'CO_time', 'TO3', 'TO3_time',  'SAT', 'SAT_time', 'HUM', 'HUM_time', 'SLHF', 'SLHF_time', 'RAIN', 'RAIN_time');
        end
    end    
    
    k = k + 1;
end

% At the end all the downloaded data are saved on the disk
save(fullfile('data', 'Atmospheric_data_from_MERRA2.mat'), 'TCWV', 'TCWV_time', 'SO2', 'SO2_time', 'DMS', 'DMS_time', 'AOT', 'AOT_time', 'CO', 'CO_time', 'TO3', 'TO3_time',  'SAT', 'SAT_time', 'HUM', 'HUM_time', 'SLHF', 'SLHF_time', 'RAIN', 'RAIN_time');


year_tba = 1980

month_tba = 12

day_tba = 26

year_tba = 1980

month_tba = 12

...

year_tba = 2016

month_tba = 4

day_tba = 8

year_tba = 2016

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



## Ionosphere data
In this session, the magnetic data from Swarm three-identical satellites are pre-processed.

Data must be downloaded from ESA's ftp dissemination service, which requires a free EO user account (you can create one here: https://eoiam-idp.eo.esa.int/) and placed in one folder of your computer were the VRE is running.

In the following example the data are inside the folder **ndir** = `E:\Swarm_data`, but **remember to replace with your own data path**  
The ESA Swarm file names appear as:  
SW_OPER_MAGA_LR_1B_20131125T110252_20131125T235959_0602_MDR_MAG_LR.cdf  
SW_OPER_MAGA_LR_1B_20201027T134456_20201027T150128_0605_MDR_MAG_LR.cdf  
...  
SW_OPER_MAGC_LR_1B_20140524T000000_20140524T235959_0602_MDR_MAG_LR.cdf


The first part of the following code selects the data in the time range defined for the analysis of the earthquake.


In [5]:
% Replace the following line with the folder in your local computer where you previously downloaded the ESA SWarm data and unzipped it.
ndir = 'E:\Swarm_data/';

EQ_time = datenum([EQ_year, EQ_month, EQ_day, EQ_hour, EQ_minute, EQ_second]);
time_start = EQ_time - day_before;
time_end = EQ_time + day_after;

files = dir([ndir, 'SW_OPER_MAG*cdf']);
files = {files.name}';

k=1;
newfiles={};
for i=1:size(files,1)
    a=files{i};
    t=datenum([str2double(a(20:23)),str2double(a(24:25)),str2double(a(26:27))]);
    if t>=time_start && t<=time_end
        newfiles(k,:)=files(i);
        k = k + 1;
    end
end


**Here we defined a function that is necessary to properly count the satellite track number of Swarm satellite data.**  
The following cell defines a function called *"dat_order_alpha"* that permits to organise the satellite data from Swarm.  
In particular, it adds a column with a number corresponding to which satellite the data belongs (1=*Alpha*, 2=*Bravo*, 3=*Charlie*) and determines the semi-orbit number.  

In [6]:
function [yn]=dat_orden_alpha(x,SAT)
Mmlat=70;% maximum and minimum latitude (outside the polar areas)
if SAT=='A';sat=1;end
if SAT=='B';sat=2;end
if SAT=='C';sat=3;end
if SAT=='S';sat=4;end
%We add 2 new colums: SAT YYYY MM DD HH MM SS TRACK-NUMBER LAT LONG R X Y Z F
y=x(abs(x(:,7))<=Mmlat,:);
nn=size(y,1);
yn=zeros(nn-1,size(y,2)+2);
if nn>1
    k=1;m=1;
    while k<=nn-1
        yn(k,:)=[sat,y(k,1:6),m,y(k,7:end)];
        if abs(y(k,7)-y(k+1,7))>.5 || abs(y(k,8)-y(k+1,8))>1;
            m=m+1;
        end
        k=k+1;
    end
end
return

In [8]:
radius_Dobrovolsky = 10.^(0.43*EQ_mag)/111;
directory_to_save_Swarm_tracks = fullfile("data", "Swarm_tracks");

i_tr = 1;
% The following line is limited to the first 10 files for quick execution.
for i=1:size(newfiles,1)
    a=newfiles{i};
    %Read the .cdf file.
    x=cdfread(fullfile(ndir,newfiles{i}), 'Variable', {'Timestamp', 'Latitude', 'Longitude', 'Radius', 'Flags_F', 'Flags_B', 'Flags_q', 'Flags_Platform', 'B_NEC', 'F'}, 'ConvertEpochToDatenum', true);%Read the .cdf.dat file.
    % Set the data in a new variable y ordering the columns and converting all the data in double variable format
    y = nan(size(x,1), 17);
    for k = 1:size(x, 1)
        y(k,:) = [double(datevec(x{k,1})), double(cell2mat(x(k,2:4))), double(cell2mat(x(k,5:7))), double(cell2mat(x(k,8))), double(cell2mat(x(k,9))'), double(cell2mat(x(k,10)))];
    end
    yn=dat_orden_alpha(y,a(12));% Include the number of track for each day
    y=yn( abs(yn(:,10)-epilon)<=(radius_Dobrovolsky+15),:);
    if size(y,1) > 50
        %Create a file name with a progressive index 
        filename_Swarm = ['Swarm_track_', num2str(i_tr, '%05d'), '.mat'];
        save(fullfile(directory_to_save_Swarm_tracks, filename_Swarm), "y", "-mat");
        i_tr = i_tr + 1;
    end
end
i_tr


Error connecting to MATLAB. Check the status of MATLAB by clicking the "Open MATLAB" button. Retry after ensuring MATLAB is running successfully

## Geomagnetic indexes
The following cells are necessary to update the geomagnetic indexes Dst and ap files.  
*Please, note that is not necessary to update these files for new case studies except it requires more recent data not previously stored in the same files.  
### Updating the Dst geomagnetic index

In [9]:
idst_old=load('dst_index2.dat');

[anno_c, mese_c, giorno_c] = datevec(now);

for anno=2000:anno_c
    for mese = 1:12
        [dst_mese, status_read] = urlread(['https://wdc.kugi.kyoto-u.ac.jp/dst_realtime/', num2str(anno), num2str(mese, '%02.0f'), '/index.html']);
        if status_read == 1
            righe=0;
            i = 1;
            ctrl = 1;
            while ctrl == 1
                if (dst_mese(i) == 10)
                    righe = righe + 1;
                end
                i = i + 1;
                if (dst_mese(i) == 10 && (dst_mese(i+1) == 'D') && (dst_mese(i+2) == 'A') && (dst_mese(i+3) == 'Y'))
                    ctrl = 0;
                end
            end
            i = i + 5;
            start_char = i;
            ctrl = 1;
            number_of_days = 1;
            while ctrl == 1
                if (dst_mese(i) == 10 && dst_mese(i+1) == '<')
                    ctrl = 0;
                end
                if (dst_mese(i) == 10)
                    number_of_days = number_of_days + 1;
                end
                i = i + 1;
            end
            dst_mese = dst_mese(start_char:i-1);
            ctrl = 0;
            k = 1;
            for i = 1:size(dst_mese, 2)
                if dst_mese(i) ~= 10
                    dst_mese_day(k) = dst_mese(i);
                    k = k + 1;
                else
                    ctrl = 1;
                end
    
                if ctrl == 1
                    k = 1;
                    dst_mese_day2 = [anno, mese, str2num(dst_mese_day(1:3)), str2num(dst_mese_day(4:7)), str2num(dst_mese_day(8:11)), str2num(dst_mese_day(12:15)), str2num(dst_mese_day(16:19)), str2num(dst_mese_day(20:23)), str2num(dst_mese_day(24:27)), str2num(dst_mese_day(28:31)), str2num(dst_mese_day(32:35)), str2num(dst_mese_day(37:40)), str2num(dst_mese_day(41:44)), str2num(dst_mese_day(45:48)), str2num(dst_mese_day(49:52)), str2num(dst_mese_day(53:56)), str2num(dst_mese_day(57:60)), str2num(dst_mese_day(61:64)), str2num(dst_mese_day(65:68)), str2num(dst_mese_day(70:73)), str2num(dst_mese_day(74:77)), str2num(dst_mese_day(78:81)), str2num(dst_mese_day(82:85)), str2num(dst_mese_day(86:89)), str2num(dst_mese_day(90:93)), str2num(dst_mese_day(94:97)), str2num(dst_mese_day(98:101))];
                    old_value = idst_old(idst_old(:,1) == anno & idst_old(:,2) == mese & idst_old(:,3) == dst_mese_day2(1,3), :);
                    if size(old_value,1) == 0
                            idst_old = [dst_mese_day2; idst_old];
                    elseif sum(old_value ~= dst_mese_day2) > 0
                        idst_old(idst_old(:,1) == anno & idst_old(:,2) == mese & idst_old(:,3) == dst_mese_day2(1,3), :) = dst_mese_day2;
                    end
                    ctrl = 0;
                end
    
            end
            
        end
    end
end



for anno=2000:anno_c
    for mese = 1:12
        [dst_mese, status_read] = urlread(['https://wdc.kugi.kyoto-u.ac.jp/dst_provisional/', num2str(anno), num2str(mese, '%02.0f'), '/index.html']);
        if status_read == 1
            righe=0;
            i = 1;
            ctrl = 1;
            while ctrl == 1
                if (dst_mese(i) == 10)
                    righe = righe + 1;
                end
                i = i + 1;
                if (dst_mese(i) == 10 && (dst_mese(i+1) == 'D') && (dst_mese(i+2) == 'A') && (dst_mese(i+3) == 'Y'))
                    ctrl = 0;
                end
            end
            i = i + 5;
            start_char = i;
            ctrl = 1;
            number_of_days = 1;
            while ctrl == 1
                if (dst_mese(i) == 10 && dst_mese(i+1) == '<')
                    ctrl = 0;
                end
                if (dst_mese(i) == 10)
                    number_of_days = number_of_days + 1;
                end
                i = i + 1;
            end
            dst_mese = dst_mese(start_char:i-1);
            ctrl = 0;
            k = 1;
            for i = 1:size(dst_mese, 2)
                if dst_mese(i) ~= 10
                    dst_mese_day(k) = dst_mese(i);
                    k = k + 1;
                else
                    ctrl = 1;
                end
    
                if ctrl == 1
                    k = 1;
                    dst_mese_day2 = [anno, mese, str2num(dst_mese_day(1:3)), str2num(dst_mese_day(4:7)), str2num(dst_mese_day(8:11)), str2num(dst_mese_day(12:15)), str2num(dst_mese_day(16:19)), str2num(dst_mese_day(20:23)), str2num(dst_mese_day(24:27)), str2num(dst_mese_day(28:31)), str2num(dst_mese_day(32:35)), str2num(dst_mese_day(37:40)), str2num(dst_mese_day(41:44)), str2num(dst_mese_day(45:48)), str2num(dst_mese_day(49:52)), str2num(dst_mese_day(53:56)), str2num(dst_mese_day(57:60)), str2num(dst_mese_day(61:64)), str2num(dst_mese_day(65:68)), str2num(dst_mese_day(70:73)), str2num(dst_mese_day(74:77)), str2num(dst_mese_day(78:81)), str2num(dst_mese_day(82:85)), str2num(dst_mese_day(86:89)), str2num(dst_mese_day(90:93)), str2num(dst_mese_day(94:97)), str2num(dst_mese_day(98:101))];
                    old_value = idst_old(idst_old(:,1) == anno & idst_old(:,2) == mese & idst_old(:,3) == dst_mese_day2(1,3), :);
                    if size(old_value,1) == 0
                            idst_old = [dst_mese_day2; idst_old];
                    elseif sum(old_value ~= dst_mese_day2) > 0
                        idst_old(idst_old(:,1) == anno & idst_old(:,2) == mese & idst_old(:,3) == dst_mese_day2(1,3), :) = dst_mese_day2;
                    end
                    ctrl = 0;
                end
    
            end
            
        end
    end
end
  


for anno=2000:anno_c
    anno
    for mese = 1:12
        [dst_mese, status_read] = urlread(['https://wdc.kugi.kyoto-u.ac.jp/dst_final/', num2str(anno), num2str(mese, '%02.0f'), '/index.html']);
        if status_read == 1
            righe=0;
            i = 1;
            ctrl = 1;
            while ctrl == 1
                if (dst_mese(i) == 10)
                    righe = righe + 1;
                end
                i = i + 1;
                if (dst_mese(i) == 10 && (dst_mese(i+1) == 'D') && (dst_mese(i+2) == 'A') && (dst_mese(i+3) == 'Y'))
                    ctrl = 0;
                end
            end
            i = i + 5;
            start_char = i;
            ctrl = 1;
            number_of_days = 1;
            while ctrl == 1
                if (dst_mese(i) == 10 && dst_mese(i+1) == '<')
                    ctrl = 0;
                end
                if (dst_mese(i) == 10)
                    number_of_days = number_of_days + 1;
                end
                i = i + 1;
            end
            dst_mese = dst_mese(start_char:i-1);
            ctrl = 0;
            k = 1;
            for i = 1:size(dst_mese, 2)
                if dst_mese(i) ~= 10
                    dst_mese_day(k) = dst_mese(i);
                    k = k + 1;
                else
                    ctrl = 1;
                end
    
                if ctrl == 1
                    k = 1;
                    dst_mese_day2 = [anno, mese, str2num(dst_mese_day(1:3)), str2num(dst_mese_day(4:7)), str2num(dst_mese_day(8:11)), str2num(dst_mese_day(12:15)), str2num(dst_mese_day(16:19)), str2num(dst_mese_day(20:23)), str2num(dst_mese_day(24:27)), str2num(dst_mese_day(28:31)), str2num(dst_mese_day(32:35)), str2num(dst_mese_day(37:40)), str2num(dst_mese_day(41:44)), str2num(dst_mese_day(45:48)), str2num(dst_mese_day(49:52)), str2num(dst_mese_day(53:56)), str2num(dst_mese_day(57:60)), str2num(dst_mese_day(61:64)), str2num(dst_mese_day(65:68)), str2num(dst_mese_day(70:73)), str2num(dst_mese_day(74:77)), str2num(dst_mese_day(78:81)), str2num(dst_mese_day(82:85)), str2num(dst_mese_day(86:89)), str2num(dst_mese_day(90:93)), str2num(dst_mese_day(94:97)), str2num(dst_mese_day(98:101))];
                    old_value = idst_old(idst_old(:,1) == anno & idst_old(:,2) == mese & idst_old(:,3) == dst_mese_day2(1,3), :);
                    if size(old_value,1) == 0
                            idst_old = [dst_mese_day2; idst_old];
                    elseif sum(old_value ~= dst_mese_day2) > 0
                        idst_old(idst_old(:,1) == anno & idst_old(:,2) == mese & idst_old(:,3) == dst_mese_day2(1,3), :) = dst_mese_day2;
                    end
                    ctrl = 0;
                end
    
            end
            
        end
    end
end
 
idst_old = sortrows(idst_old);
for i=1:24
    idst_old = idst_old(idst_old(:, 3+i) ~= 9999, :);
end


file_output = fopen('dst_index2.dat', 'w');
for j = 1:size(idst_old, 1)
   fprintf(file_output, '%d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d\n', idst_old(j,:));
end
fclose(file_output);


Error connecting to MATLAB. Check the status of MATLAB by clicking the "Open MATLAB" button. Retry after ensuring MATLAB is running successfully

### Updating the ap geomagnetic index
The following cell defines a simple function to convert the kp to ap according to standard tables.

In [ ]:
function [ap] = kp2ap(kp)

if (kp == '0o') ap = 0; end;
if (kp == '0+') ap = 2; end;
if (kp == '1-') ap = 3; end;
if (kp == '1o') ap = 4; end;
if (kp == '1+') ap = 5; end;
if (kp == '2-') ap = 6; end;
if (kp == '2o') ap = 7; end;
if (kp == '2+') ap = 9; end;
if (kp == '3-') ap = 12; end;
if (kp == '3o') ap = 15; end;
if (kp == '3+') ap = 18; end;
if (kp == '4-') ap = 22; end;
if (kp == '4o') ap = 27; end;
if (kp == '4+') ap = 32; end;
if (kp == '5-') ap = 39; end;
if (kp == '5o') ap = 48; end;
if (kp == '5+') ap = 56; end;
if (kp == '6-') ap = 67; end;
if (kp == '6o') ap = 80; end;
if (kp == '6+') ap = 94; end;
if (kp == '7-') ap = 111; end;
if (kp == '7o') ap = 132; end;
if (kp == '7+') ap = 154; end;
if (kp == '8-') ap = 179; end;
if (kp == '8o') ap = 207; end;
if (kp == '8+') ap = 236; end;
if (kp == '9-') ap = 300; end;
if (kp == '9o') ap = 400; end;

return

The following cell updates the ap geomagnetic index in the file `ap_index2.dat`.

In [ ]:
iap=load('ap_index2.dat');

anno_di_partenza = 2000;

iap = [];
[anno_corrente, mese_corrente] = datevec(now);
mesi_totali = (anno_corrente-anno_di_partenza)*12+mese_corrente-2;
for mese1 = 1:mesi_totali
    anno_da_controllare = anno_di_partenza + fix((mese1-1)/12)
    mese_da_controllare = mese1 - (anno_da_controllare-anno_di_partenza)*12;
    anno_testo = num2str(anno_da_controllare);
    kp_mese = urlread(['ftp://ftp.gfz-potsdam.de/pub/home/obs/kp-ap/tab/kp', anno_testo(3), anno_testo(4), num2str(mese_da_controllare, '%02.0f'),'.tab']);
    righe=0;
    ctrl = 1;
    for i=2:size(kp_mese, 2)
        if (kp_mese(i) == 10) && ctrl == 1 righe = righe + 1; end
        if (kp_mese(i) == 10) && (kp_mese(i-1) == 10) ctrl = 0; end
    end
    righe = righe - 2;
    ap_table = [];

    i=0;
    for j = 1:righe
        anno = (kp_mese(1+i)-48)*10 + kp_mese(2+i)-48+2000;
        mese = (kp_mese(3+i)-48)*10 + kp_mese(4+i)-48;
        giorno = (kp_mese(5+i)-48)*10 + kp_mese(6+i)-48;
        ap_table(j,1:3) = [anno, mese, giorno];
        ap_table(j,4) = kp2ap(char([kp_mese(9+i), kp_mese(10+i)]));
        ap_table(j,5) = kp2ap(char([kp_mese(12+i), kp_mese(13+i)]));
        ap_table(j,6) = kp2ap(char([kp_mese(15+i), kp_mese(16+i)]));        
        ap_table(j,7) = kp2ap(char([kp_mese(18+i), kp_mese(19+i)]));
        ap_table(j,8) = kp2ap(char([kp_mese(22+i), kp_mese(23+i)]));
        ap_table(j,9) = kp2ap(char([kp_mese(25+i), kp_mese(26+i)]));        
        ap_table(j,10) = kp2ap(char([kp_mese(28+i), kp_mese(29+i)]));
        ap_table(j,11) = kp2ap(char([kp_mese(31+i), kp_mese(32+i)]));
        i = 33 + i;
        a = kp_mese(i);
        while (a ~= 10)
            i = i + 1;
            a = kp_mese(i);    
        end
    end
    iap = [iap; ap_table];
end

kp_mese_precedente = urlread('https://www-app3.gfz-potsdam.de/kp_index/pqlyymm.tab');
% kp_mese_precedente = urlread('ftp://ftp.gfz-potsdam.de/pub/home/obs/kp-ap/tab/kp1903.tab');

righe=0;
for i=1:size(kp_mese_precedente, 2)
    if (kp_mese_precedente (i) == 10)
        righe = righe + 1;
    end;
    
end;
righe = righe - 1;
% righe = righe - 3; %%% da togliere per aggirnamento standard

ap_table = [];

i=0;
% for j = 1:righe
for j = 1:righe
    
    anno = (kp_mese_precedente(1+i)-48)*10 + kp_mese_precedente(2+i)-48+2000;
    mese = (kp_mese_precedente(3+i)-48)*10 + kp_mese_precedente(4+i)-48;
    giorno = (kp_mese_precedente(5+i)-48)*10 + kp_mese_precedente(6+i)-48;
    ap_table(j,1:3) = [anno, mese, giorno];
    ap_table(j,4) = kp2ap(char([kp_mese_precedente(9+i), kp_mese_precedente(10+i)]));
    ap_table(j,5) = kp2ap(char([kp_mese_precedente(12+i), kp_mese_precedente(13+i)]));
    ap_table(j,6) = kp2ap(char([kp_mese_precedente(15+i), kp_mese_precedente(16+i)]));        
    ap_table(j,7) = kp2ap(char([kp_mese_precedente(18+i), kp_mese_precedente(19+i)]));
    ap_table(j,8) = kp2ap(char([kp_mese_precedente(22+i), kp_mese_precedente(23+i)]));
    ap_table(j,9) = kp2ap(char([kp_mese_precedente(25+i), kp_mese_precedente(26+i)]));        
    ap_table(j,10) = kp2ap(char([kp_mese_precedente(28+i), kp_mese_precedente(29+i)]));
	ap_table(j,11) = kp2ap(char([kp_mese_precedente(31+i), kp_mese_precedente(32+i)]));
    i = 33 + i;
    a = kp_mese_precedente(i);
    while (a ~= 10)
        i = i + 1;
        a = kp_mese_precedente(i);    
    end
end

if aggiornamento_completo == 0
    file_output = fopen('ap_index2.dat', 'a');
    ctrl = 0;
    mese = iap(end, 2);
    ultimo_giorno = iap(end, 3) + 1;
    if (mese == ap_table(1,2))
        ctrl = 1;
        for j = ultimo_giorno:righe
            fprintf(file_output, '\n%d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d', ap_table(j,1:3), ap_table(j,4), ap_table(j,4),  ap_table(j,4),  ap_table(j,5),  ap_table(j,5),  ap_table(j,5),  ap_table(j,6),  ap_table(j,6),  ap_table(j,6),  ap_table(j,7),  ap_table(j,7),  ap_table(j,7),  ap_table(j,8),  ap_table(j,8),  ap_table(j,8),  ap_table(j,9),  ap_table(j,9),  ap_table(j,9),  ap_table(j,10),  ap_table(j,10),  ap_table(j,10),  ap_table(j,11),  ap_table(j,11),  ap_table(j,11));
        end
    end
    iap=load('ap_index2.dat');
elseif aggiornamento_completo == 1
    iap = [iap; ap_table];
end

kp_mese_corrente = urlread('https://www-app3.gfz-potsdam.de/kp_index/qlyymm.tab');

righe=0;
for i=1:size(kp_mese_corrente, 2)
    if (kp_mese_corrente(i) == 10)
        righe = righe + 1;
    end;
    
end;
righe = righe - 2;

ap_table = [];

i=0;
for j = 1:righe
    
    anno = (kp_mese_corrente(1+i)-48)*10 + kp_mese_corrente(2+i)-48+2000;
    mese = (kp_mese_corrente(3+i)-48)*10 + kp_mese_corrente(4+i)-48;
    giorno = (kp_mese_corrente(5+i)-48)*10 + kp_mese_corrente(6+i)-48;
    ap_table(j,1:3) = [anno, mese, giorno];
    ap_table(j,4) = kp2ap(char([kp_mese_corrente(9+i), kp_mese_corrente(10+i)]));
    ap_table(j,5) = kp2ap(char([kp_mese_corrente(12+i), kp_mese_corrente(13+i)]));
    ap_table(j,6) = kp2ap(char([kp_mese_corrente(15+i), kp_mese_corrente(16+i)]));        
    ap_table(j,7) = kp2ap(char([kp_mese_corrente(18+i), kp_mese_corrente(19+i)]));
    ap_table(j,8) = kp2ap(char([kp_mese_corrente(22+i), kp_mese_corrente(23+i)]));
    ap_table(j,9) = kp2ap(char([kp_mese_corrente(25+i), kp_mese_corrente(26+i)]));        
    ap_table(j,10) = kp2ap(char([kp_mese_corrente(28+i), kp_mese_corrente(29+i)]));
	ap_table(j,11) = kp2ap(char([kp_mese_corrente(31+i), kp_mese_corrente(32+i)]));
    i = 33 + i;
    a = kp_mese_corrente(i);
    while (a ~= 10)
        i = i + 1;
        a = kp_mese_corrente(i);    
    end
end

iap = [iap; ap_table];
file_output = fopen('ap_index2.dat', 'w');
for j = 1:size(iap,1)
    fprintf(file_output, '\n%d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d %d', iap(j,1:3), iap(j,4), iap(j,4),  iap(j,4),  iap(j,5),  iap(j,5),  iap(j,5),  iap(j,6),  iap(j,6),  iap(j,6),  iap(j,7),  iap(j,7),  iap(j,7),  iap(j,8),  iap(j,8),  iap(j,8),  iap(j,9),  iap(j,9),  iap(j,9),  iap(j,10),  iap(j,10),  iap(j,10),  iap(j,11),  iap(j,11),  iap(j,11));
end

fclose(file_output);